In [1]:
import numpy as np
from pathlib import Path
from collections.abc import Mapping, Sequence
from typing import Iterable, Tuple, Any, Union
def paths_to_numpy_int32_including_arrays(root: Any) -> Iterable[Path]:
    """
    Like `paths_to_numpy_int32`, but also drills into numpy arrays.
    Array indices are appended to the path as integers.
    """
    def _walk(obj: Any, path: Path):
        if isinstance(obj, np.int32):
            yield path
            return

        if isinstance(obj, Mapping):
            for k, v in obj.items():
                yield from _walk(v, path + (k,))
            return

        if isinstance(obj, Sequence) and not isinstance(obj, (str, bytes, bytearray)):
            # Handle python lists/tuples
            for i, v in enumerate(obj):
                yield from _walk(v, path + (i,))
            return

        # Handle numpy arrays element-wise if dtype is int32
        if isinstance(obj, np.ndarray):
            if obj.dtype == np.int32:
                # iterate with ndenumerate to get multi-dimensional indices
                for idx, val in np.ndenumerate(obj):
                    # idx is a tuple of ints; append as a single step if you prefer
                    # but flattening them keeps path navigation straightforward:
                    yield path + idx
            else:
                # if not int32 dtype, still dive into object arrays cautiously
                if obj.dtype == object:
                    for idx, val in np.ndenumerate(obj):
                        yield from _walk(val, path + idx)

    return _walk(root, ())